 # Hückel π simples em Piridina (C5H5N) 
# 
 **Montar** a Hamiltoniana 6×6 do anel π da piridina (1 = N; 2…6 = C),
 diagonalizar com `numpy.linalg.eigh` (mesma abordagem dos sistemas de molas),
 ocupar 6 elétrons (2 por MO), identificar HOMO/LUMO, calcular E_gap,
 e obter populações por sítio `q_i` e ordens de ligação π `p_ij` (apenas 1º vizinhos).
 
 **Parâmetros (exatamente os do enunciado)**  
 α_C = 0.00, α_N = +0.50, β_CC = −1.00, β_CN = −0.90.  
 Energias em unidades de β_CC (β_CC = −1 define a unidade).

 Saídas: CSVs em `tables/` e figuras em `figures/` que foram usadas no Artigo

In [18]:

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

try:
    import pandas as pd  # só para salvar tabelas com cabeçalhos bonitos
    USE_PANDAS = True
except Exception:
    USE_PANDAS = False

# Pastas de saída (criadas automaticamente)
BASE = Path.cwd()
FIG = BASE / "figures"
TAB = BASE / "tables"
FIG.mkdir(exist_ok=True)
TAB.mkdir(exist_ok=True)

print("Saídas serão gravadas em:")
print("  FIG:", FIG)
print("  TAB:", TAB)



Saídas serão gravadas em:
  FIG: c:\Users\Gabriel Simarelli\Desktop\projeto\Artigo\figures
  TAB: c:\Users\Gabriel Simarelli\Desktop\projeto\Artigo\tables



## 1) Definição do modelo e Hamiltoniana H (6×6)
 - Base local |i⟩ com 6 sítios p_z (1 = N; 2…6 = C)
 - Vizinhança de 1ºs vizinhos: (1,2), (2,3), (3,4), (4,5), (5,6), (6,1)
 - Diagonais: H_11 = α_N; H_ii = α_C (i = 2…6)
 - Fora da diagonal **apenas** 1ºs vizinhos: β_CN quando envolve N–C; β_CC para C–C

In [19]:
# Parâmetros fixos (exatamente os solicitados)
ALPHA_C = 0.00
ALPHA_N = +0.50
BETA_CC = -1.00
BETA_CN = -0.90

# Topologia (1-based no enunciado; vamos guardar assim e converter para 0-based no código)
NEIGHBORS_1BASED = [(1,2), (2,3), (3,4), (4,5), (5,6), (6,1)]

SITE_TYPES = {1: "N", 2: "C", 3: "C", 4: "C", 5: "C", 6: "C"}

def build_hamiltonian():
    """Constroi a Hamiltoniana 6×6 do modelo de Hückel simples para a piridina."""
    H = np.zeros((6,6), dtype=float)

    # Diagonais
    for i in range(6):
        if i == 0:      # índice 0 ↔ sítio 1 (N)
            H[i,i] = ALPHA_N
        else:           # sítios 2..6 (C)
            H[i,i] = ALPHA_C

    # Fora da diagonal: somente 1ºs vizinhos
    for (a,b) in NEIGHBORS_1BASED:
        i, j = a-1, b-1
        # tipo de ligação (C–N ou C–C)
        types = (SITE_TYPES[a], SITE_TYPES[b])
        beta = BETA_CN if ("N" in types and "C" in types) else BETA_CC
        H[i,j] = beta
        H[j,i] = beta
    return H

H = build_hamiltonian()
print("Hamiltoniana H (unid. β_CC):\n", H)

Hamiltoniana H (unid. β_CC):
 [[ 0.5 -0.9  0.   0.   0.  -0.9]
 [-0.9  0.  -1.   0.   0.   0. ]
 [ 0.  -1.   0.  -1.   0.   0. ]
 [ 0.   0.  -1.   0.  -1.   0. ]
 [ 0.   0.   0.  -1.   0.  -1. ]
 [-0.9  0.   0.   0.  -1.   0. ]]


 **Explicação (etapa 1)**  
 Modelo de Hückel simples: mantemos apenas integrais de Coulomb (diagonais) e de
 ressonância (off-diagonal) entre orbitais p_z de **primeiros vizinhos**.
 O nitrogênio (sítio 1) tem α_N = +0.50 (mais eletronegativo), carbonos α_C = 0.00.
 As ligações C–C levam β_CC = −1.00 (define a unidade de energia) e as C–N
 β_CN = −0.90. Nada além dos 1ºs vizinhos entra na matriz (exatamente como pedido).


## 2) Diagonalização e ocupação eletrônica (6 elétrons π)
 - Usamos `numpy.linalg.eigh(H)` (matriz simétrica → autovalores em ordem crescente)
 - Ocupamos 6 elétrons: 2 por MO → 3 orbitais mais baixos preenchidos
 - Identificamos HOMO (último ocupado), LUMO (primeiro desocupado) e calculamos E_gap

In [20]:
E, C = np.linalg.eigh(H)              # autovalores (E) e autovetores (colunas de C)
occ = np.zeros_like(E)
occ[:3] = 2.0                         # 3 MOs ocupadas com 2 elétrons cada (total 6)

homo = 2                              # índice 0-based → 3º nível
lumo = 3
Egap = E[lumo] - E[homo]

print("Autovalores (E_μ, unid. β_CC):\n", E)
print(f"HOMO = nível {homo+1}, LUMO = nível {lumo+1}, Egap = {Egap:.4f}")

Autovalores (E_μ, unid. β_CC):
 [-1.88719505 -1.         -0.75738947  1.          1.11916415  2.02542037]
HOMO = nível 3, LUMO = nível 4, Egap = 1.7574


 **Explicação (etapa 2)**  
 A rotina `eigh` é a mesma abordagem numérica das matrizes simétricas discutidas
 nas aulas (ex.: molas acopladas). Cada autovetor (coluna de `C`) define uma MO π,
 e `C[i,μ]` é o coeficiente da MO μ no sítio i. Com 6 elétrons, preenchemos os
 3 níveis de menor energia (2 elétrons por nível). O **HOMO** é o 3º nível, o
 **LUMO** o 4º; a diferença **E_gap = E_LUMO − E_HOMO** será usada na discussão.

## 3) Observáveis: populações `q_i` e ordens de ligação π `p_ij`
 Fórmulas (somando sobre MOs **ocupadas**):
 - População por sítio:  q_i = Σ 2 |c_i^(μ)|²
 - Ordem de ligação π:   p_ij = Σ 2 c_i^(μ) c_j^(μ)   (apenas 1º vizinhos)

In [21]:
def site_populations(C, occ):
    """q_i = sum_occ 2*|c_i^μ|^2"""
    return (np.abs(C)**2 @ occ)

def bond_orders(C, occ):
    """p_ij = sum_occ 2*c_i^μ*c_j^μ para 1ºs vizinhos (pares 1-based)."""
    P = {}
    for (a,b) in NEIGHBORS_1BASED:
        i, j = a-1, b-1
        val = float(2.0 * np.sum(C[i,:] * C[j,:] * occ))
        P[(a,b)] = val
    return P

q = site_populations(C, occ)
P = bond_orders(C, occ)

print("Populações q_i (i=1..6):", q, "  soma =", np.sum(q))
print("Ordens p_ij (1ºs vizinhos):", P)

Populações q_i (i=1..6): [0.78242832 1.08447569 0.99541016 1.05779999 0.99541016 1.08447569]   soma = 5.999999999999997
Ordens p_ij (1ºs vizinhos): {(1, 2): 1.2933236501906034, (2, 3): 1.3551617164501186, (3, 4): 1.3210385922467638, (4, 5): 1.3210385922467638, (5, 6): 1.355161716450118, (6, 1): 1.2933236501906034}



 **Explicação (etapa 3)**  
 `q_i` mede a densidade eletrônica π acumulada no sítio i pelas MOs ocupadas;
 a soma **deve ser ≈ 6** (número total de elétrons π).  
 `p_ij` mede o compartilhamento eletrônico π entre vizinhos (apenas 1ºs vizinhos),
 útil para **comparar C–N vs C–C** na piridina. Valores maiores indicam mais
 caráter de ligação π; diferenças entre C–N e C–C devem aparecer devido a α_N e β_CN.

 ## 4) Funções para gerar Figuras (PNG)
 - **Figura 1**: esquema do anel numerado (1 = N)  
 - **Figura 2**: espectro (stick plot), com HOMO/LUMO anotados  
 - **Figuras 6 e 7**: mapas “bolha” de HOMO e LUMO (raio ∝ |c_i|; marcador indica fase)

In [35]:
import math
from matplotlib.lines import Line2D

def _ring_coords(n=6, R=1.0):
    """Coordenadas de n pontos igualmente espaçados num círculo (começando no +y, sentido horário)."""
    angs = [math.pi/2 - 2*math.pi*k/n for k in range(n)]
    xs = [R*math.cos(a) for a in angs]
    ys = [R*math.sin(a) for a in angs]
    return xs, ys

def fig1_scheme(savepath):
    xs, ys = _ring_coords(6, 1.0)
    plt.figure()
    # anel (liga pontos em ciclo)
    for i in range(6):
        j = (i+1) % 6
        plt.plot([xs[i], xs[j]], [ys[i], ys[j]])
    # marca nós
    plt.scatter(xs, ys)
    # numeração (1..6) e destaque do N
    for i in range(6):
        label = f"{i+1}" if i != 0 else "1 (N)"
        plt.text(xs[i], ys[i], label, ha='center', va='center')
    plt.axis('equal'); plt.axis('off')
    plt.title("Figura 1 — Piridina numerada (1 = N)")
    plt.tight_layout()
    plt.savefig(savepath, dpi=200); plt.close()

def fig2_spectrum(E, homo, lumo, savepath):
    plt.figure()
    y0, y1 = 0.0, 1.0
    for k, Ek in enumerate(E):
        plt.vlines(Ek, y0, y1)
    plt.text(E[homo], 1.05, "HOMO", ha="center", rotation=90)
    plt.text(E[lumo], 1.05, "LUMO", ha="center", rotation=90)
    plt.ylim(0, 1.2)
    plt.xlim(np.min(E)-0.5, np.max(E)+0.5)
    plt.yticks([])
    plt.xlabel("Energia (unid. β_CC)")
    plt.title("Figura 2 — Espectro de Hückel (stick) com HOMO/LUMO")
    plt.tight_layout()
    plt.savefig(savepath, dpi=200); plt.close()



def fig_bubble(vec, title, savepath):
    xs, ys = _ring_coords(6, 1.0)
    mags = np.abs(vec)
    sizes = 2000 * (mags / mags.max())**2 if mags.max() > 0 else np.ones_like(mags)*100
    pos = [i for i in range(6) if vec[i] >= 0]
    neg = [i for i in range(6) if vec[i] < 0]

    plt.figure()
    # anel
    for i in range(6):
        j = (i+1) % 6
        plt.plot([xs[i], xs[j]], [ys[i], ys[j]])

    # marcadores por fase (tamanho original)
    plt.scatter([xs[i] for i in pos], [ys[i] for i in pos],
                s=[sizes[i] for i in pos], marker='o', label='fase +')
    plt.scatter([xs[i] for i in neg], [ys[i] for i in neg],
                s=[sizes[i] for i in neg], marker='x', label='fase −')

    # numeração (1 = N)
    for i in range(6):
        lab = f"{i+1}" if i != 0 else "1 (N)"
        plt.text(xs[i], ys[i], lab, ha='center', va='center')

    # legenda com símbolos 75% menores
    legend_elements = [
        Line2D([0],[0], marker='o', color='w', label='fase +',
               markerfacecolor='C0', markersize=18),  # 75% do padrão (~8)
        Line2D([0],[0], marker='x', color='C1', label='fase −',
               markersize=18)                         # idem
    ]
    plt.legend(handles=legend_elements, loc='upper right')

    plt.axis('equal'); plt.axis('off')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(savepath, dpi=200)
    plt.close()


 **Explicação (etapa 4 — figuras)**  
 - **Esquema numerado (Fig. 1)**: mostra a convenção 1 = N, essencial para ler tabelas/coeficientes.  
 - **Espectro (Fig. 2)**: cada linha vertical é um autovalor E_μ; HOMO/LUMO anotados.
 A leitura facilita a identificação de lacuna E_gap e a ocupação.  
 - **Mapas “bolha” (Fig. 6 e Fig. 7)**: área ∝ |c_i|² evidencia onde a MO concentra densidade;
 o marcador `o/x` distingue a **fase** (sinal) do coeficiente — importante para entender
 ordens de ligação (produtos c_i c_j nos vizinhos).

 ## 5) Exportar Tabelas (CSV) e salvar Figuras (PNG)
 - Tabela 2: autovalores e ocupações (com flags HOMO/LUMO e E_gap no rodapé)
 - Tabela 4: populações q_i por sítio (verificação Σ_i q_i ≈ 6)
 - Tabela 5: ordens p_ij em 1ºs vizinhos (separar C–N e C–C)
 - Fig. 1, Fig. 2, Fig. 6 (HOMO), Fig. 7 (LUMO)

In [36]:
# %%
def save_tables_and_figures():
    # --- Tabelas ---
    idx = np.arange(1, 7)
    if USE_PANDAS:
        import pandas as pd
        df_eig = pd.DataFrame({
            "mu": idx,
            "E_mu (β_CC)": E,
            "occupation": occ,
            "is_HOMO": [i==(homo+1) for i in idx],
            "is_LUMO": [i==(lumo+1) for i in idx],
        })
        df_pop = pd.DataFrame({
            "site": idx,
            "type": [SITE_TYPES[i] for i in idx],
            "q_i": q
        })
        # bond orders
        bonds = []
        for (a,b), val in P.items():
            btype = "C–N" if ("N" in (SITE_TYPES[a], SITE_TYPES[b])) else "C–C"
            bonds.append({"bond (i,j)": f"({a},{b})", "type": btype, "p_ij": val})
        df_bond = pd.DataFrame(bonds)

        df_eig.to_csv(TAB / "eigenvalues.csv", index=False)
        df_pop.to_csv(TAB / "populations.csv", index=False)
        df_bond.to_csv(TAB / "bond_orders.csv", index=False)

        # guarda Egap em arquivo txt simples para referência rápida
        (TAB / "gap_info.txt").write_text(f"Egap (β_CC) = {Egap:.6f}\n", encoding="utf-8")
    else:
        # fallback sem pandas
        np.savetxt(TAB / "eigenvalues.csv",
                   np.c_[idx, E, occ],
                   delimiter=",", header="mu,E_mu(β_CC),occupation", comments="")
        np.savetxt(TAB / "populations.csv",
                   np.c_[idx, [0 if i!=1 else 1 for i in idx], q],
                   delimiter=",", header="site,isN(1=N),q_i", comments="")
        with open(TAB / "bond_orders.csv", "w", encoding="utf-8") as f:
            f.write("bond(i,j),type,p_ij\n")
            for (a,b), val in P.items():
                btype = "C–N" if ("N" in (SITE_TYPES[a], SITE_TYPES[b])) else "C–C"
                f.write(f"({a},{b}),{btype},{val}\n")
        (TAB / "gap_info.txt").write_text(f"Egap (β_CC) = {Egap:.6f}\n", encoding="utf-8")

    # --- Figuras ---
    fig1_scheme(FIG / "fig1_scheme_pyridine.png")
    fig2_spectrum(E, homo, lumo, FIG / "fig2_spectrum_stick.png")
    fig_bubble(C[:, homo], "Figura 6 — Mapa HOMO (raio ∝ |c_i|; fase em marcador)", FIG / "fig6_map_HOMO.png")
    fig_bubble(C[:, lumo], "Figura 7 — Mapa LUMO (raio ∝ |c_i|; fase em marcador)", FIG / "fig7_map_LUMO.png")

save_tables_and_figures()
print("Arquivos gerados em:\n -", FIG, "\n -", TAB)

Arquivos gerados em:
 - c:\Users\Gabriel Simarelli\Desktop\projeto\Artigo\figures 
 - c:\Users\Gabriel Simarelli\Desktop\projeto\Artigo\tables


 **Explicação (etapa 5 — exportação e interpretação das imagens)**  
 - **Tabela eigenvalues.csv**: lista E_μ (β_CC), ocupações (2 ou 0) e flags HOMO/LUMO.  
 - **Tabela populations.csv**: `q_i` por sítio; verifique que Σ q_i ≈ 6 (fechado).  
 - **Tabela bond_orders.csv**: `p_ij` para (1,2), (2,3), ..., (6,1), com coluna `type` (C–N ou C–C).  
 - **Figura 1**: apenas a numeração (1 = N) para referência visual no texto.  
 - **Figura 2**: “sticks” nos autovalores; HOMO/LUMO anotados — a leitura visual ajuda a reportar **E_gap**.  
 - **Fig. 6 e 7**: bolhas maiores onde |c_i| é maior; `o` indica fase positiva e `x` fase negativa.
   A alternância de fase entre vizinhos é chave para o sinal de `p_ij` (produto c_i c_j).